In [1]:
# Importando Pacotes
from google.cloud import bigquery
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time 
from datetime import date, datetime
import pandas as pd
import os
from google.oauth2 import service_account
from pandas.io import gbq


In [3]:
# Funções de captura 01 
# Empresa da reclamacao
def empresa_reclamacao():
    empresa = driver.find_element_by_xpath('//*[@id="hero"]/div[2]/div/div[2]/div[1]/h2').get_attribute('title')
    return empresa
# Titulo reclamacao
def titulo_reclamacao():
    titulo = driver.find_element(By.XPATH,'//*[@id="__next"]/div[1]/div[1]/div[4]/main/div/div[2]/div[1]/div[1]/div[3]/h1').text
    return titulo
# Local da reclamacao
def local_reclamacao():
    local = driver.find_element(By.XPATH,'//*[@id="__next"]/div[1]/div[1]/div[4]/main/div/div[2]/div[1]/div[1]/div[3]/div[1]/div[2]/div[1]/span').text                                     
    return local
# Data inicial dareclamacao
def datainicial_reclamacao():
    datainicial = driver.find_element(By.XPATH,'//*[@id="__next"]/div[1]/div[1]/div[4]/main/div/div[2]/div[1]/div[1]/div[3]/div[1]/div[2]/div[2]/span').text
                                                
    return datainicial
# Data final da reclamacao
def datafinal_reclamacao():
    datafinal = driver.find_element(By.XPATH,'//*[@id="__next"]/div[1]/div[1]/div[4]/main/div/div[2]/div[1]/div[2]/div[2]/div[1]/div/span').text
    return datafinal
# Status da reclamacao
def status_reclamacao():
    status = driver.find_element(By.XPATH,'//*[@id="__next"]/div[1]/div[1]/div[4]/main/div/div[2]/div[1]/div[2]/div[2]/div[2]/div[1]/div/div/span').text
    return status
# Vfn da reclamcao
def vfn_reclamacao():
    vfn = driver.find_element(By.XPATH,'//*[@id="__next"]/div[1]/div[1]/div[4]/main/div/div[2]/div[1]/div[2]/div[2]/div[2]/div[2]/div/div').text
    return vfn
# Nota da reclamacao
def nota_reclamacao():
    nota = driver.find_element(By.XPATH,'//*[@id="__next"]/div[1]/div[1]/div[4]/main/div/div[2]/div[1]/div[2]/div[2]/div[2]/div[3]/div/div').text
    return nota

In [4]:
############# Extraindo Métricas de Avaliação ################
# Credenciais Google Cloud
credentials = service_account.Credentials.from_service_account_file(filename='/home/alex/apps/drivers/ContaServicoBigQuery.json', 
                                                                    scopes=["https://www.googleapis.com/auth/cloud-platform"])
client = bigquery.Client(credentials=credentials)
# Query SQL
sql = """
SELECT
CONCAT(A.link,"/lista-reclamacoes/") AS link
FROM `carbide-legend-331517.ReclameAquiStage.empresas_ranking` AS A
WHERE posicao = 2
ORDER BY posicao
"""
# Executando Query
link_lojas = client.query(sql).to_dataframe()
link_lojas = link_lojas['link'].values
# Criando link das reclamações
link_reclamacoes = link_lojas

# Criando dataframe para armazenar informações das reclamacoes
info_reclamacoes = pd.DataFrame(columns = ['id_empresa','idreclamacao','empresa','link','titulo','local','data_inicial','data_final','status','vfn','nota']) 

 # Criando data frame para armazenar erros
info_erros = pd.DataFrame(columns =['iderro','id_empresa','idreclamacao','empresa','link_pag','link'])

# Carregando driver
# driver = webdriver.Chrome('/home/alex/Documentos/Unifal/ReclameAquiTCC/codigos/drive/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument("--disable-logging")
options.add_argument("--log-level=3")
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.maximize_window()
# Valor para fazer id_empresa
id_empresa = 2

# Contador para id_reclamacao
idreclamacao = 0

# Contador de erros
iderro = 0


## Estrutura de repetição para inserir dados das reclamações
for link_reclamacao in link_reclamacoes:
    # Chave identificadora da empresa
    id_empresa = id_empresa + 1
    # Data de limite para coleta dados
    data_atual = time.strftime("%d/%m/%Y")
    data_reclamacao = datetime.strptime(data_atual,"%d/%m/%Y")
    # # Add -3 months to a given datetime object
    # n = -3
    # data_limite = pd.to_datetime(data_atual) + pd.DateOffset(months=n)
    # print(data_limite)
    # data_limite = data_limite.strftime("%d/%m/%Y")
    # data_limite = time.strptime(data_limite, "%d/%m/%Y")
    # print(data_limite)
    data_limite = "08/04/2022"
    data_limite = datetime.strptime(data_limite,"%d/%m/%Y")
    print(data_limite)
    # Contador de páginas acessadas
    p = 998
    # Estutura de repetição para ir mudando as páginas
    while p < 1010:
        p = p +1
        for reclamacao in range(1,11):
            try:
                idreclamacao = idreclamacao + 1
                link_pag = str(link_reclamacao) + '?pagina=' + str(p) + '&status=EVALUATED'
                driver.get(link_pag)
                time.sleep(5)       
                driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()        
                time.sleep(2)
                driver.delete_all_cookies()
                # Busca empresa da reclamação
                empresa = empresa_reclamacao()                                              
                # Armaezena link da reclamacao
                link = driver.find_element_by_xpath('//*[@id="__next"]/div[1]/div[1]/div[4]/main/section[2]/div[2]/div[2]/div[' + str(reclamacao) + ']/a').get_attribute('href')                     
                # Entra na reclamação avaliada
                driver.find_element_by_xpath('//*[@id="__next"]/div[1]/div[1]/div[4]/main/section[2]/div[2]/div[2]/div[' + str(reclamacao) + ']/a/h4').click()        
                time.sleep(5)
                # Titulo reclamacao
                titulo = titulo_reclamacao()              
                # Local da reclamação
                local = local_reclamacao()
                # Data inicial reclamacao
                data_inicial = datainicial_reclamacao()                                   
                # Data final
                data_final = datafinal_reclamacao()                                               
                # Status reclamacao
                status = status_reclamacao()
                # Voltaria fazer negócio
                vfn = vfn_reclamacao()
                # Nota
                nota = nota_reclamacao()               
                # Convertendo data_inicial para validação
                data_inicial = data_inicial[:10]
                data_reclamacao = datetime.strptime(data_inicial,"%d/%m/%Y")
                # Convertendo data_finall para validação
                data_final = data_final[:10]
                # Clausúla para continuar o loop
                if data_reclamacao < data_limite:
                    break   
                # Inserindo dados no dataframe
                info_reclamacoes = pd.DataFrame([[id_empresa,idreclamacao,empresa,link,titulo,local,data_inicial,data_final,status,vfn,nota]],columns = info_reclamacoes.columns).append(info_reclamacoes)
            except:
                iderro = iderro +1
                info_erros = pd.DataFrame([[iderro,id_empresa,idreclamacao,empresa,link_pag,link]],columns = info_erros.columns).append(info_erros)
            pass                
time.sleep(15)
# fechando drive
driver.close() 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST chromedriver version for 101.0.4951 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_linux64.zip
[WDM] - Driver has been saved in cache [/home/alex/.wdm/drivers/chromedriver/linux64/101.0.4951.41]


2022-04-08 00:00:00


/tmp/ipykernel_3088/1217253692.py:72: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
/tmp/ipykernel_3088/2141956480.py:4: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  empresa = driver.find_element_by_xpath('//*[@id="hero"]/div[2]/div/div[2]/div[1]/h2').get_attribute('title')
/tmp/ipykernel_3088/1217253692.py:78: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  link = driver.find_element_by_xpath('//*[@id="__next"]/div[1]/div[1]/div[4]/main/section[2]/div[2]/div[2]/div[' + str(reclamacao) + ']/a').get_attribute('href')
/tmp/ipykernel_3088/1217253692.py:80: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//

In [5]:
info_reclamacoes.shape

(68, 11)

In [6]:
info_erros.shape

(52, 6)

In [13]:
# Passando arquivo Big Query
gbq.to_gbq(dataframe = info_reclamacoes,credentials=credentials, 
                                destination_table='ReclameAquiStage.info_reclamacoes', 
                                if_exists='append',
                                table_schema=[{'name': 'id_empresa', 'type': 'INTEGER'},
                                            {'name': 'idreclamacao', 'type': 'INTEGER'},
                                            {'name': 'empresa', 'type': 'STRING'},
                                            {'name': 'link', 'type': 'STRING'},
                                            {'name': 'titulo', 'type': 'STRING'},
                                            {'name': 'local', 'type': 'STRING'},
                                            {'name': 'data_inicial', 'type': 'STRING'},
                                            {'name': 'data_final', 'type': 'STRING'},
                                            {'name': 'status', 'type': 'STRING'},
                                            {'name': 'vfn', 'type': 'STRING'},
                                            {'name': 'nota', 'type': 'STRING'}])
# Passando arquivo Big Query
gbq.to_gbq(dataframe = info_erros,credentials=credentials, 
                                destination_table='ReclameAquiStage.info_erros', 
                                if_exists='append',
                                table_schema=[{'name': 'iderro', 'type': 'INTEGER'},
                                            {'name': 'id_empresa', 'type': 'INTEGER'},
                                            {'name': 'idreclamacao', 'type': 'INTEGER'},
                                            {'name': 'empresa', 'type': 'STRING'},
                                            {'name': 'link_pag', 'type': 'STRING'},
                                         {'name': 'link', 'type': 'STRING'}])

In [21]:
############# Extraindo Métricas de Avaliação ################
# Credenciais Google Cloud
credentials = service_account.Credentials.from_service_account_file(filename='/home/alex/apps/drivers/ContaServicoBigQuery.json', 
                                                                    scopes=["https://www.googleapis.com/auth/cloud-platform"])
client = bigquery.Client(credentials=credentials)
# Query SQL
sql = """
SELECT 
  id_empresa,
  idreclamacao,
  empresa,
  link  
FROM `carbide-legend-331517.ReclameAquiStage.info_erros` 
"""
df = client.query(sql).to_dataframe()
lista = df.values.tolist()

In [26]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import TimeoutException
//*[@id="islrg"]/div[1]/div[1]/a[1]/div[1]/img
//*[@id="yDmH0d"]/div[2]/c-wiz/c-wiz/div[2]/div/div/div[1]/div[2]/span/div[1]/div[1]/div[1]/a[1]/div[1]/img

In [32]:
# Carregando o driver
options = webdriver.ChromeOptions()
options.add_argument("--disable-logging")
options.add_argument("--log-level=3")
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.maximize_window()              

# Criando dataframe para armazenar informações das reclamacoes
info_reclamacoes = pd.DataFrame(columns = ['id_empresa','idreclamacao','empresa','link','titulo','local','data_inicial','data_final','status','vfn','nota']) 

 # Criando data frame para armazenar erros
info_erros = pd.DataFrame(columns =['iderro','id_empresa','tipo','idreclamacao','empresa','link'])

# Contador de erros
iderro = 0
# for id_empresa, idreclamacao, empresa, link in lista:
try:
        # link_pag = str(link_reclamacao) + '?pagina=' + str(p) + '&status=EVALUATED' 
        link_pag = 'https://www.reclameaqui.com.br/mapfre-seguros/seguro-estendido-que-nao-funciona_UOtyuNQ4EOCglhL0'         
        # # Armaezena link da reclamacao
        # link = link_pag                     
        driver.get(link_pag)
        # Titulo reclamacao
        titulo = driver.find_element_by_css_selector('.berwWw').text         
        # Local da reclamação
        local = driver.find_element_by_css_selector('.erEgDU').text  
        # Data inicial reclamacao
        data_inicial = driver.find_element_by_css_selector('.ePqLRr').text                                 
        # Data final
        data_final = driver.find_element_by_css_selector('.gdZHpb .ipwWvs').text   
        # Status
        status = driver.find_element_by_css_selector('.gfOjuF').text
        # Vfn 
        vfn = driver.find_element_by_css_selector('.brUBl').text
        # Nota
        nota = driver.find_element_by_css_selector('.kcxYqf~ .kcxYqf+ .kcxYqf .ceUcTc').text                                                          
        # Convertendo data_inicial para validação
        data_inicial = data_inicial[:10]
        # Convertendo data_finall para validação
        data_final = data_final[:10]
        info_reclamacoes = pd.DataFrame([[id_empresa,idreclamacao,empresa,link,titulo,local,data_inicial,data_final,status,vfn,nota]],columns = info_reclamacoes.columns).append(info_reclamacoes)
except NoSuchElementException:
        tipo = 'Erro de captura'
        iderro = iderro + 1
        info_erros = pd.DataFrame([[iderro,id_empresa,tipo,idreclamacao,empresa,link]],columns = info_erros.columns).append(info_erros)
        pass
except TimeoutException as e:
        tipo = 'Erro de tempo resposta'
        iderro = iderro + 1
        info_erros = pd.DataFrame([[iderro,id_empresa,tipo,idreclamacao,empresa,link]],columns = info_erros.columns).append(info_erros)
        pass
except WebDriverException as web_driver_exception:
        tipo = 'Erro de conexão'
        iderro = iderro + 1
        info_erros = pd.DataFrame([[iderro,id_empresa,tipo,idreclamacao,empresa,link]],columns = info_erros.columns).append(info_erros)
        pass
except:
        tipo = 'Sem identificação'
        iderro = iderro + 1
        info_erros = pd.DataFrame([[iderro,id_empresa,tipo,idreclamacao,empresa,link]],columns = info_erros.columns).append(info_erros)
        pass                       
# fechando drive
driver.close() 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST chromedriver version for 101.0.4951 google-chrome
[WDM] - Driver [/home/alex/.wdm/drivers/chromedriver/linux64/101.0.4951.41/chromedriver] found in cache
/tmp/ipykernel_22103/2413340235.py:25: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  titulo = driver.find_element_by_css_selector('.berwWw').text
/tmp/ipykernel_22103/2413340235.py:27: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  local = driver.find_element_by_css_selector('.erEgDU').text
/tmp/ipykernel_22103/2413340235.py:29: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  data_inicial = driver.find_element_by_css_selector('.ePqLRr').text
/tmp/ipykernel_2

In [16]:
info_erros['id_empresa'].replace({4: 2},inplace=True)

In [9]:
info_reclamacoes.head()

,id_empresa,idreclamacao,empresa,link,titulo,local,data_inicial,data_final,status,vfn,nota
0,3,1520,Bradesco Seguros,https://www.reclameaqui.com.br/bradesco-seguro...,Atraso na Entrega remédio Químioterapico,Guarulhos - SP,17/04/2022,22/04/2022,Resolvido,Sim,10
0,3,1519,Bradesco Seguros,https://www.reclameaqui.com.br/bradesco-seguro...,Cobrança indevida,São Paulo - SP,18/04/2022,28/04/2022,Resolvido,Sim,10
0,3,1518,Bradesco Seguros,https://www.reclameaqui.com.br/bradesco-seguro...,Aplicativo Bradesco Saúde não funciona e pagam...,São Paulo - SP,18/04/2022,04/05/2022,Resolvido,Sim,9
0,3,1516,Bradesco Seguros,https://www.reclameaqui.com.br/bradesco-seguro...,Estorno / CANCELAMENTO de Seguro de vida.,Floriano - PI,18/04/2022,26/04/2022,Resolvido,Sim,10
0,3,1515,Bradesco Seguros,https://www.reclameaqui.com.br/bradesco-seguro...,Quero Cancelar e Estornar o meu seguro de vida,Jerumenha - PI,18/04/2022,26/04/2022,Resolvido,Sim,9


In [10]:
info_erros.head()

,iderro,id_empresa,idreclamacao,empresa,link_pag,link
0,563,3,1517,Bradesco Seguros,https://www.reclameaqui.com.br/empresa/bradesc...,https://www.reclameaqui.com.br/bradesco-seguro...
0,562,3,1513,Bradesco Seguros,https://www.reclameaqui.com.br/empresa/bradesc...,https://www.reclameaqui.com.br/bradesco-seguro...
0,561,3,1508,Bradesco Seguros,https://www.reclameaqui.com.br/empresa/bradesc...,https://www.reclameaqui.com.br/bradesco-seguro...
0,560,3,1504,Bradesco Seguros,https://www.reclameaqui.com.br/empresa/bradesc...,https://www.reclameaqui.com.br/bradesco-seguro...
0,559,3,1499,Bradesco Seguros,https://www.reclameaqui.com.br/empresa/bradesc...,https://www.reclameaqui.com.br/bradesco-seguro...


In [8]:
gbq.to_gbq(dataframe = info_reclamacoes,credentials=credentials, 
                                destination_table='ReclameAquiStage.info_reclamacoes', 
                                if_exists='append',
                                table_schema=[{'name': 'id_empresa', 'type': 'INTEGER'},
                                            {'name': 'idreclamacao', 'type': 'INTEGER'},
                                            {'name': 'empresa', 'type': 'STRING'},
                                            {'name': 'link', 'type': 'STRING'},
                                            {'name': 'titulo', 'type': 'STRING'},
                                            {'name': 'local', 'type': 'STRING'},
                                            {'name': 'data_inicial', 'type': 'STRING'},
                                            {'name': 'data_final', 'type': 'STRING'},
                                            {'name': 'status', 'type': 'STRING'},
                                            {'name': 'vfn', 'type': 'STRING'},
                                            {'name': 'nota', 'type': 'STRING'}])

In [9]:
gbq.to_gbq(dataframe = info_erros,credentials=credentials, 
                                destination_table='ReclameAquiStage.info_erros', 
                                if_exists='append',
                                table_schema=[{'name': 'iderro', 'type': 'INTEGER'},
                                            {'name': 'id_empresa', 'type': 'INTEGER'},
                                            {'name': 'idreclamacao', 'type': 'INTEGER'},
                                            {'name': 'empresa', 'type': 'STRING'},
                                            {'name': 'link_pag', 'type': 'STRING'},
                                            {'name': 'link', 'type': 'STRING'}])